<a href="https://colab.research.google.com/github/Ashu10051998/Face-Emotion-Detection/blob/main/Face_Emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url= https://www.dropbox.com/s/si11cws2pyho1bp/archive.zip

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [3]:
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential


In [4]:
import tensorflow as tf
from pathlib import Path

tf.random.set_seed(4)

In [5]:
!wget https://www.dropbox.com/s/si11cws2pyho1bp/archive.zip

--2021-09-16 15:08:17--  https://www.dropbox.com/s/si11cws2pyho1bp/archive.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/si11cws2pyho1bp/archive.zip [following]
--2021-09-16 15:08:17--  https://www.dropbox.com/s/raw/si11cws2pyho1bp/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc9f3244d6f995762bfe8e6ce7a.dl.dropboxusercontent.com/cd/0/inline/BWSkP7z8m4CENVb0jL08Uy8tQdOoFrTK2r24WMg-Hjy8zeKvAoosAFGtpKj7TVc4pUnWI4Gk3fECiOJvFHpRv9-uUT2hV39wpV6b4MLcRn54LL-0G8cK74VuilIvbY9Cm0L48_4WMjbxstFQUJqP885u/file# [following]
--2021-09-16 15:08:17--  https://ucc9f3244d6f995762bfe8e6ce7a.dl.dropboxusercontent.com/cd/0/inline/BWSkP7z8m4CENVb0jL08Uy8tQdOoFrTK2r24WMg-Hjy8zeKvAoosAFGtpKj7TVc4pUnWI4Gk3fECiOJvFHpRv9-uUT

In [6]:
!unzip -q "/content/archive.zip"

In [9]:
train_path = Path("/content/train")
test_path = Path("/content/test")

In [10]:
train_image_paths = list(train_path.glob("*/*"))
train_image_paths = list(map(lambda x : str(x) , train_image_paths))

train_image_paths[:10]

['/content/train/neutral/Training_57074106.jpg',
 '/content/train/neutral/Training_4679433.jpg',
 '/content/train/neutral/Training_99339110.jpg',
 '/content/train/neutral/Training_26220773.jpg',
 '/content/train/neutral/Training_24212024.jpg',
 '/content/train/neutral/Training_70623999.jpg',
 '/content/train/neutral/Training_34289515.jpg',
 '/content/train/neutral/Training_69405202.jpg',
 '/content/train/neutral/Training_31924948.jpg',
 '/content/train/neutral/Training_61929223.jpg']

In [11]:
# Getting their respective labels 

def get_label(image_path):
    return image_path.split("/")[-2]

train_image_labels = list(map(lambda x : get_label(x) , train_image_paths))
train_image_labels[:10]

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral']

In [12]:
from sklearn.preprocessing import LabelEncoder 

In [13]:
le = LabelEncoder()

In [18]:
train_labels = le.fit_transform(train_image_labels)
train_labels[:10]

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [19]:
train_labels = tf.keras.utils.to_categorical(train_image_labels)
train_labels[:10]

array([[0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)